### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/03/20 08:54:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 08:54:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/banco_original_enriched'))

### Care Site

In [4]:
df_fact_relationship = (df_input
                        .select('person_id','person_id_infant').distinct()
                        .withColumn('domain_concept_id_1', F.lit(4248584))
                        .withColumn('fact_id_1', F.col('person_id'))
                        .withColumn('domain_concept_id_2', F.lit(4285883))
                        .withColumn('fact_id_2', F.col('person_id_infant'))
                        .withColumn('relationship_concept_id', F.lit(4277283))
                       ).select(F.col('domain_concept_id_1').cast('integer'), 
                                F.col('fact_id_1').cast('integer'),
                                F.col('domain_concept_id_2').cast('integer'), 
                                F.col('fact_id_2').cast('integer'),
                                F.col('relationship_concept_id').cast('integer'))


df_fact_relationship.count()

24683852

### Salvando no Postgresql

In [5]:
write_to_postgresl(df_fact_relationship, tb_name='fact_relationship', write_mode='append')

'Total time: 0:04:39.048408 and - Total rows: 24683852 - Total columns: 5'

## Deprecated

### Salvando CSV 

In [ ]:
# df_fact_relationship.repartition(1).write.csv('/data/IDAF/SHARED/omop_tables/fact_relationship.csv', header = True, mode='overwrite')

## SQL de insertion

In [ ]:
# -- CREATE TABLE public.fact_relationship (
# -- 			domain_concept_id_1 integer NOT NULL,
# -- 			fact_id_1 integer NOT NULL,
# -- 			domain_concept_id_2 integer NOT NULL,
# -- 			fact_id_2 integer NOT NULL,
# -- 			relationship_concept_id integer NOT NULL );

# -- CREATE TABLE public.fact_relationship_pyspark (
# -- 			domain_concept_id_1 varchar,
# -- 			fact_id_1 varchar,
# -- 			domain_concept_id_2 varchar,
# -- 			fact_id_2 varchar,
# -- 			relationship_concept_id varchar);
			
# insert into public.fact_relationship (
# domain_concept_id_1,
# fact_id_1,
# domain_concept_id_2,
# location_id,
# fact_id_2,
# relationship_concept_id 
# )
# SELECT 
# domain_concept_id_1,
# fact_id_1,
# domain_concept_id_2,
# location_id,
# fact_id_2,
# relationship_concept_id 
# 	FROM public.fact_relationship_pyspark;


